# SUBWORD EMBEDDING WITH 100K DATA

Here, 4 different models investigated.

- Model 1: basic Word2Vec algortihm. Inputs are complete words without splitting



- Model 2: Word2Vec algorithm where inputs are splitted based on some common suffixess and prefixess.



- Model 3: Word2Vec algorithm where inputs with more than 4 chcarcter, splitted based on compounds structure but also can be effective to detect suffixes (not prefix)



- Model 4: Combination of Model 2 and Model 3

# 1. IMPORTING THE LIBRARIES 

In [1]:
from gensim.models.word2vec import Word2Vec
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.corpora import Dictionary
from statistics import mean 
from gensim.models.fasttext import FastText

# 2. READING THE FILES

## 2.1 Reading the Dataset

I downloaded text file with 3OOK sentences from the The Wortschatz Leipzig project (https://wortschatz.uni-leipzig.de/en/download). In this part I am reading and creating the dataset. 

In [2]:
#openning the text file 
text= open("eng-uk_web-public_2018_300K-sentences.txt")

#reading the file 
data = text.readlines()

In [3]:
#number of sentences in the data set is 
number_of_sentences = len(data)
print("Number of Sentences in the Dataset =",number_of_sentences)

Number of Sentences in the Dataset = 300000


## 2.2 Reading the Word Similarity Dataset 1

Here, I have choosen wordsim353 as Word Similarity Dataset 1.(http://alfonseca.org/eng/research/wordsim353.html).

In [4]:
#openning the text file 
similarity_text_one = open('wordsim_similarity_goldstandard.txt' )


#reading the file 
similarity_dataset_one = similarity_text_one.readlines()

Now,  I haveto preprocess the Word Simlarity Dataset 1 to compare with trained dataset in the future. To do this.

First I will split the each line into words, make the value float and also transform the values into (-1,1). Because in the Word Similarity dataset , values are between 0-10, and in the Word"vec is between -1 to -1.

In [5]:

#for convenience, splitting the each line into tokens
similarity_one = []

for i in similarity_dataset_one:
    splitted= i.split()
    
    #tranlating the similarity value to float and normalizing by dividing 10 and adding-1
    splitted[2] = float(splitted[2])/5 -1
    
    similarity_one.append(splitted)
    
#example: 
print(similarity_one[1])

['tiger', 'tiger', 1.0]


## 2.3 Reading the Word Similarity Dataset 2 (with Rare Words)

Here I used SynEngl.txt that was provided in the Learnweb.

In [6]:
#openning the text file 
similarity_text_two = open('SynEngl.txt' )


#reading the file 
similarity_dataset_two = similarity_text_two.readlines() 

Now,  I have preprocess the word simlarity dataset 2 to compare with trained dataset in the future. To do this, first I will split the each line into words, translate the binary values "-,+" into "0,1". 

In [7]:

#for convenience, splitting the each line into words
similarity_two = []

for j in similarity_dataset_two:
    splitted= j.split()
    if splitted[2]=="-":
        splitted[2] = 0
    elif splitted[2]=="+":
        splitted[2]=1  
    similarity_two.append(splitted)
    
#example: 
print(similarity_two[104])

['allotments', 'parceling', 1]


# 4. TOKENIZATION: SPLITTING THE SENTENCES INTO WORDS

For the simplicity, i used gensim.utils.simple_preprocess function.

In [8]:
#creating a list of tokens of all sentences 
all_tokens = [] 

for i in range(0,100000):
    tokens = gensim.utils.simple_preprocess(data[i])
    all_tokens.append(tokens)   

In [9]:
#example: 101 th sentence's token
print("101th sentence = \n", data[100])
print("101 th sentence's Token = \n", all_tokens[100])

101th sentence = 
 101	£5,000 to people with one or more with-profits annuities from Equitable Life which started to be paid before 1 September 1992.

101 th sentence's Token = 
 ['to', 'people', 'with', 'one', 'or', 'more', 'with', 'profits', 'annuities', 'from', 'equitable', 'life', 'which', 'started', 'to', 'be', 'paid', 'before', 'september']


# 5. CREATING AN ACCURACY FUNCTION

To able to compare the accuracy of the Model 1, 2 and 3 , I need a accuracy calculator function. In this part,  I will create it.

In [10]:
# S: Word Similarity Dataset
# W: All traiend words
# M: Model

def accuracy(S,W,M):
    all_d = []
    for i in S:
        if i[0] in W and i[1] in W:
            model_similarity= M.wv.similarity(i[0], i[1])
            d = abs(abs(model_similarity) -i[2])
            all_d.append(d)
            
    print("Number Of Words That Accuracy Calculated = ",len(all_d),"\nAccuracy = ", 1-mean(all_d))
    

# 6. MODEL 1  (WITHOUT SPLITTING)

When I train the all tokens, I used Word2Vec algorithm.

## 6.1 Training the Model 1

Now, we will train the Model 1 with 32 workers.

In [11]:
#training the model 1 with 32 workers
model_one = Word2Vec(all_tokens,workers=32)

## 6.2 Example: Embedding Vector of a Word in Model 1

After traiend the 1M sentences, we obtained the word embedding for eact trained word.

In [12]:
#example: word embeddings of "supermarkets"
model_one["supermarket"]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([-8.68099090e-03,  3.33725870e-01, -1.77786320e-01,  5.60775585e-02,
       -7.86022171e-02,  1.71825346e-02,  8.42858180e-02,  8.65730271e-02,
        2.03533038e-01, -2.01631010e-01, -5.58173545e-02,  1.77839212e-02,
        4.03980650e-02, -1.05171651e-01, -3.74714658e-02, -2.79189348e-01,
       -2.56004661e-01, -2.05921397e-01,  5.63053191e-02,  1.77175581e-01,
       -1.65620446e-01, -4.64087799e-02, -1.25890315e-01,  3.26763131e-02,
        4.13272411e-01, -7.44047239e-02, -6.68330789e-02,  8.44212770e-02,
       -8.56756642e-02,  7.93641880e-02,  2.78564300e-02,  1.82377696e-01,
        1.11022696e-01, -1.68368131e-01, -8.18734169e-02, -2.30612393e-04,
       -1.10410921e-01,  8.84325728e-02,  2.06919783e-03, -6.41171262e-02,
       -1.46977693e-01, -4.61323485e-02, -9.52041820e-02,  2.93663472e-01,
        2.28438854e-01, -2.38785014e-01, -1.00261287e-03, -8.09088275e-02,
        1.58772171e-01,  3.13179195e-01,  1.79688781e-01, -1.09745607e-01,
       -1.11449637e-01, -

## 6.3  Example: Top 10 Similar to a Word 

In [13]:
# the most 10 similar words to discussion model 1
model_one.wv.most_similar('criminal', topn=10)  # get other similar words

[('fraud', 0.8875191807746887),
 ('statutory', 0.8653087615966797),
 ('gdpr', 0.8477980494499207),
 ('statements', 0.8477556109428406),
 ('pensions', 0.8408083915710449),
 ('licensing', 0.835369884967804),
 ('disclosure', 0.8338068127632141),
 ('headings', 0.8321937322616577),
 ('freedoms', 0.83077073097229),
 ('regulation', 0.8276646137237549)]

## 6.5 Example: Similarity Between Two Words

In [14]:
print("Similarity Super and Supermarkets = ", model_one.wv.similarity('super', 'supermarket'))

Similarity Super and Supermarkets =  0.6957606


# 7 MODEL 2 (WITH SPLITTING BASED ON PREFIX AND SUFFIX)

In this model, I created a function which splits the raw tokens (words). The created function takes into consideration the some most frequent suffixes and prefixes.

## 7.1 Splitting Function  (Words Based on Suffix and Prefix)

In [15]:
def splitting(sentence):
    liste = []
    for word in sentence:    
        n = len(word)
        
        if n>3:
            #SUFFIXES
            if word[n-2:n]=="ed":
                a = word[:-2]
                liste.append(word)
                liste.append(a)
                #liste.append("ed")
           
                
            elif word[n-3:n]=="ing":
                liste.append(word)
                a = word[:-3]
                liste.append(a)
                #liste.append("ing")
            
              
                
            elif word[-1] =="s":
                liste.append(word)
                a= word[:-1] 
                liste.append(a)
                #liste.append("s")
           
                
            elif word[n-4:n]=="able":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("able")
            
             
                
            elif word[n-5:n]=="tion":
                liste.append(word)
                a = word[:-5]
                liste.append(a)
                #liste.append("ation")
            
                
            elif word[n-4:n]=="hood":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("hood")
              
             
                
            elif word[n-2:n]=="er":
                liste.append(word)
                a = word[:-2]
                liste.append(a)
                #liste.append("er")
            
                
            elif word[n-2:n]=="ly":
                liste.append(word)
                a = word[:-2]
                liste.append(a)
                liste.append("ly")
               # sentence.remove(word)
                
            elif word[n-3:n]=="ist":
                liste.append(word)
                a = word[:-3]
                liste.append(a)
                #liste.append("ist")
           
                
                 
            elif word[n-2:n]=="es":
                liste.append(word)
                a = word[:-2]
                liste.append(a)
               # liste.append("es")
               
                
                 
            elif word[n-3:n]=="let":
                liste.append(word)
                a = word[:-3]
                liste.append(a)
                #liste.append("let")
            
            elif word[n-4:n]=="ness":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("let")
                
            
            elif word[n-4:n]=="ment":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("let")
     
            elif word[n-4:n]=="ment":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("less") 
                
            elif word[n-2:n]=="er":
                liste.append(word)
                a = word[:-2]
                liste.append(a)
                #liste.append("less") 
            
                
            elif word[n-4:n]=="ible":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("able")
                
                  
            elif word[n-3:n]=="ate":
                liste.append(word)
                a = word[:-3]
                liste.append(a)
                #liste.append("able")
                
                  
            elif word[n-2:n]=="fy":
                liste.append(word)
                a = word[:-2]
                liste.append(a)
                #liste.append("less")
                
             #PREFIXES      
            elif word[0:2]=="un":
                liste.append(word)
                a = word[2:n]
                liste.append(a)
                #liste.append("less")
                
            elif word[0:3]=="non":
                liste.append(word)
                a = word[3:n]
                liste.append(a)
                #liste.append("less")
            
            elif word[0:3]=="dis":
                liste.append(word)
                a = word[3:n]
                liste.append(a)
                #liste.append("less")
                
            elif word[0:4]=="anti":
                liste.append(word)
                a = word[4:n]
                liste.append(a)
                #liste.append("less")
                
                   
            elif word[0:3]=="sub":
                liste.append(word)
                a = word[3:n]
                liste.append(a)
                #liste.append("less")
                
            elif word[0:5]=="under":
                liste.append(word)
                a = word[5:n]
                liste.append(a)
                liste.append("under")
                
            elif word[0:5]=="super":
                liste.append(word)
                a = word[5:n]
                liste.append(a)
                liste.append("super")

            else:
                liste.append(word)          
              
    return liste

## 7.2 Splitting The Words

In [16]:
splitted_tokens = []
modified_dataset = all_tokens

for s in modified_dataset:
    splitted_tokens.append(splitting(s))

## 7.3 Training the Model 2 

In [17]:
model_two = Word2Vec(splitted_tokens,workers=32)

# 8. MODEL 3 (WITH SPLITTING BASED ON COMPOUNDS)

## 8.1 Creating Base Vocabulary Dataset

In [18]:
trained_words_model_one = list(model_one.wv.vocab) 
trained_words_model_two = list(model_two.wv.vocab)

vocabulary = set(trained_words_model_one + trained_words_model_two)

## 8.2 Splitting Function (Based on Compunds)

In [19]:
def splitting_two(sentence):
    liste=[]
    for word in sentence:
        n = len(word)
        #chossing the words with more than 4 character
        if n>4:
            for i in range(4,n):
                if word[0:i] in vocabulary:
                    liste.append(word[0:i])
                    if word[i:n] in trained_words_model_one:
                        liste.append(word[i:n]) 
                    
        liste.append(word)
    return liste
       
                

## 8.3 Splitting

In [20]:
splitted_tokens_two = []
modified_dataset_second = all_tokens

for s in modified_dataset_second:
    splitted_tokens_two.append(splitting_two(s))

In [21]:
splitted_tokens_two

[['leg', 'and', 'extend', 'ed', 'extended', 'bikini', 'line'],
 ['return',
  'conn',
  'connect',
  'ion',
  'connection',
  'help',
  'helps',
  'to',
  'avoid',
  'foam',
  'ing',
  'foaming',
  'of',
  'the',
  'media',
  'while',
  'tank',
  'is',
  'being',
  'fill',
  'ed',
  'filled'],
 ['of',
  'the',
  'money',
  'has',
  'been',
  'seiz',
  'ed',
  'seize',
  'seized',
  'from',
  'criminal',
  'criminals',
  'under',
  'the',
  'proce',
  'eds',
  'proceed',
  'proceeds',
  'of',
  'act',
  'and',
  'the',
  'other',
  'has',
  'come',
  'from',
  'commun',
  'community',
  'safe',
  'ty',
  'safety',
  'budget'],
 ['angu',
  'angus',
  'paid',
  'the',
  'tax',
  'due',
  'on',
  'aug',
  'more',
  'than',
  'six',
  'month',
  'months',
  'late',
  'so',
  'he',
  'will',
  'pay',
  'furth',
  'er',
  'further',
  'penalty',
  'which',
  'is',
  'anoth',
  'er',
  'another'],
 ['cotton',
  'with',
  'two',
  'extra',
  'long',
  'hand',
  'les',
  'handl',
  'es',
  'handl

## 8.3 Training the Model 3

In [22]:
model_three = Word2Vec(splitted_tokens_two,workers=32)

# 9. MODEL 4 (MIXTURED of MODEL 1 and MODEL 2)

## 9.1 Splitting Function

In [24]:
def splitting_mix(sentence):
    liste=[]
    for word in sentence:
        n = len(word)
        #chossing the words with more than 4 character
        if n>4:
            for i in range(4,n):
                if word[0:i] in vocabulary:
                    liste.append(word[0:i])
                    if word[i:n] in trained_words_model_one:
                        liste.append(word[i:n]) 
        elif n<5:
            if word[-1] =="s":
                a= word[:-1] 
                liste.append(a)
            elif word[n-2:n]=="es":
                a = word[:-2]
                liste.append(a)
        liste.append(word)
        
        
        
    return liste
                  

## 9.2 Splitting

In [25]:
splitted_tokens_mix = []
modified_dataset_mix = all_tokens

for s in modified_dataset_mix:
    splitted_tokens_mix.append(splitting_mix(s))

In [26]:
splitted_tokens_mix 

[['leg', 'and', 'extend', 'ed', 'extended', 'bikini', 'line'],
 ['return',
  'conn',
  'connect',
  'ion',
  'connection',
  'help',
  'helps',
  'to',
  'avoid',
  'foam',
  'ing',
  'foaming',
  'of',
  'the',
  'media',
  'while',
  'tank',
  'i',
  'is',
  'being',
  'fill',
  'ed',
  'filled'],
 ['of',
  'the',
  'money',
  'ha',
  'has',
  'been',
  'seiz',
  'ed',
  'seize',
  'seized',
  'from',
  'criminal',
  'criminals',
  'under',
  'the',
  'proce',
  'eds',
  'proceed',
  'proceeds',
  'of',
  'act',
  'and',
  'the',
  'other',
  'ha',
  'has',
  'come',
  'from',
  'commun',
  'community',
  'safe',
  'ty',
  'safety',
  'budget'],
 ['angu',
  'angus',
  'paid',
  'the',
  'tax',
  'due',
  'on',
  'aug',
  'more',
  'than',
  'six',
  'month',
  'months',
  'late',
  'so',
  'he',
  'will',
  'pay',
  'furth',
  'er',
  'further',
  'penalty',
  'which',
  'i',
  'is',
  'anoth',
  'er',
  'another'],
 ['cotton',
  'with',
  'two',
  'extra',
  'long',
  'hand',
  'les

In [27]:
#exampe:
print(data[40],"\n")
print(splitted_tokens_mix[40])

41	£1 spent on a cheeseburger could buy one kilo of sweet potatoes, two kilos of carrots, or two and half kilos of pasta.
 

['spent', 'on', 'cheese', 'burger', 'cheeseburger', 'could', 'buy', 'one', 'kilo', 'of', 'sweet', 'potato', 'es', 'potatoe', 'potatoes', 'two', 'kilo', 'kilos', 'of', 'carr', 'carrot', 'carrots', 'or', 'two', 'and', 'half', 'kilo', 'kilos', 'of', 'past', 'pasta']


## 9.3 Training the Model 4

In [28]:
model_mix = Word2Vec(splitted_tokens_mix,workers=32)

# 9. COMPARISON BETWEEN MODEL 1, 2 , 3, 4

## 9.1 Number of Trained Words on Each Models

In [29]:
# all trained words in model 1 and model 2
trained_words_model_one = list(model_one.wv.vocab) #model 1
trained_words_model_two = list(model_two.wv.vocab) #model 2
trained_words_model_three = list(model_three.wv.vocab) #model 3
trained_words_model_mix = list(model_mix.wv.vocab) #model 4

print("Number of Trained Words in Model 1 =",len(trained_words_model_one),"\n")
print("Number of Trained Words in Model 2 =",len(trained_words_model_two), "\n")
print("Number of Trained Words in Model 3 =",len(trained_words_model_three), "\n")
print("Number of Trained Words in Model 4 =",len(trained_words_model_mix), "\n")

Number of Trained Words in Model 1 = 17125 

Number of Trained Words in Model 2 = 19999 

Number of Trained Words in Model 3 = 20435 

Number of Trained Words in Model 4 = 20594 



## 9.2 Accuracy Score on Word Similary Dataset 1

In [30]:
#calculation of the accuracy score for Word Similarity Dataset 1
print("Model 1: \n")
accuracy(similarity_one,trained_words_model_one,model_one)

Model 1: 

Number Of Words That Accuracy Calculated =  165 
Accuracy =  0.43707555363864614


In [31]:
print("Model 2 \n")
accuracy(similarity_one,trained_words_model_two,model_two)

Model 2 

Number Of Words That Accuracy Calculated =  169 
Accuracy =  0.4763884066466749


In [32]:
print("Model 3 \n")
accuracy(similarity_one,trained_words_model_three,model_three)

Model 3 

Number Of Words That Accuracy Calculated =  172 
Accuracy =  0.5664709880102512


In [33]:
print("Model 4 \n")
accuracy(similarity_one,trained_words_model_mix,model_mix)

Model 4 

Number Of Words That Accuracy Calculated =  172 
Accuracy =  0.5715703894716883


## 9.3 Accuracy Score on Word Similary Dataset 2

In [34]:
#calculation of the accuracy score for Word Similarity Dataset 1
print("Model 1: \n")
accuracy(similarity_two,trained_words_model_one,model_one)

Model 1: 

Number Of Words That Accuracy Calculated =  247 
Accuracy =  0.581502997772655


In [35]:
print("Model 2: \n")
accuracy(similarity_two,trained_words_model_two,model_two)

Model 2: 

Number Of Words That Accuracy Calculated =  254 
Accuracy =  0.562284222540013


In [36]:
print("Model 3: \n")
accuracy(similarity_two,trained_words_model_three,model_three)

Model 3: 

Number Of Words That Accuracy Calculated =  254 
Accuracy =  0.5043581861445284


In [37]:
print("Model 4: \n")
accuracy(similarity_two,trained_words_model_mix,model_mix)

Model 4: 

Number Of Words That Accuracy Calculated =  254 
Accuracy =  0.49550319849110147


## 9.4 Some Word Pairs Similarities 

In [38]:
print("Similarity(Super,Supermarket):\n")
print(" Model 1 = ", model_one.wv.similarity('super', 'supermarket'))
print(" Model 2 = ", model_two.wv.similarity('super', 'supermarket'))
print(" Model 3 = ", model_three.wv.similarity('super', 'supermarket'))
print(" Model 4 = ", model_mix.wv.similarity('super', 'supermarket'))

Similarity(Super,Supermarket):

 Model 1 =  0.6957606
 Model 2 =  0.72487926
 Model 3 =  0.7809027
 Model 4 =  0.75700206


In [39]:
print("Similarity(Super,Supermarkets):\n")
print(" Model 1 = ", model_one.wv.similarity('super', 'supermarkets'))
print(" Model 2 = ", model_two.wv.similarity('super', 'supermarkets'))
print(" Model 3 = ", model_three.wv.similarity('super', 'supermarkets'))
print(" Model 4 = ", model_mix.wv.similarity('super', 'supermarkets'))

Similarity(Super,Supermarkets):

 Model 1 =  0.6425464
 Model 2 =  0.5887278
 Model 3 =  0.6922886
 Model 4 =  0.68990016


In [40]:
print("Similarity(citizen,citizens):\n")
print(" Model 1 = ", model_one.wv.similarity('citizen', 'citizens'))
print(" Model 2 = ", model_two.wv.similarity('citizen', 'citizens'))
print(" Model 3 = ", model_three.wv.similarity('citizen', 'citizens'))
print(" Model 4 = ", model_mix.wv.similarity('citizen', 'citizens'))

Similarity(citizen,citizens):

 Model 1 =  0.74518514
 Model 2 =  0.9594349
 Model 3 =  0.9287307
 Model 4 =  0.92439634


In [41]:
print("Similarity(Comfort, Comfortable)\n")
print(" Model 1 = ", model_one.wv.similarity('comfort', 'comfortable'))
print(" Model 2 = ", model_two.wv.similarity('comfort', 'comfortable'))
print(" Model 3 = ", model_three.wv.similarity('comfort', 'comfortable'))
print(" Model 4 = ", model_mix.wv.similarity('comfort', 'comfortable'))

Similarity(Comfort, Comfortable)

 Model 1 =  0.70579994
 Model 2 =  0.9488038
 Model 3 =  0.7987852
 Model 4 =  0.8122554


In [42]:
print("Similarity(ear, ears)\n")
print(" Model 1  = ", model_one.wv.similarity('ear', 'ears'))
print(" Model 2 = ", model_two.wv.similarity('ear', 'ears'))
print(" Model 3 = ", model_three.wv.similarity('ear', 'ears'))
print(" Model 4 = ", model_mix.wv.similarity('ear', 'ears'))

Similarity(ear, ears)

 Model 1  =  0.8068266
 Model 2 =  0.68438566
 Model 3 =  0.3964045
 Model 4 =  0.79276925


In [43]:
print("Similarity(discussion, discussionss)\n")
print(" Model 1  = ", model_one.wv.similarity('discussion', 'discussions'))
print(" Model 2  = ", model_two.wv.similarity('discussion', 'discussions'))
print(" Model 3 = ", model_three.wv.similarity('discussion', 'discussions'))
print(" Model 4 = ", model_mix.wv.similarity('discussion', 'discussions'))

Similarity(discussion, discussionss)

 Model 1  =  0.78610355
 Model 2  =  0.9037983
 Model 3 =  0.5668821
 Model 4 =  0.5426762
